# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df['Humidity'] = pd.to_numeric(city_data_df['Humidity'], errors= 'coerce')

#dropping rows with negative humidity values. could not confirm with open weather documentation or cust serv why values existed.
city_data_df = city_data_df[city_data_df['Humidity']>= 0]
city_data_df.dropna(subset= ['Humidity'], inplace=True)


# Display sample data
print(city_data_df['Humidity'].min())
city_data_df.head(20)

0.08


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,benghazi,32.1167,20.0667,14.43,14.43,99,5.28,LY,1738794437
1,1,mount gambier,-37.8333,140.7667,20.12,20.12,10,2.60,AU,1738794717
3,3,port-aux-francais,-49.3500,70.2167,7.39,7.39,99,8.84,TF,1738794719
4,4,grytviken,-54.2811,-36.5092,1.62,1.62,98,4.36,GS,1738794720
5,5,shenjiamen,29.9576,122.2980,3.65,3.65,6,3.40,CN,1738794721
8,8,hun,29.1268,15.9477,13.53,13.53,99,2.76,LY,1738794724
9,9,ribeira grande,38.5167,-28.7000,18.22,17.65,75,5.36,PT,1738794725
12,12,maine soroa,13.2114,12.0241,20.67,20.67,6,4.08,NE,1738794728
14,14,ouled mimoun,34.9047,-1.0339,6.61,6.61,28,0.80,DZ,1738794730
15,15,selfoss,63.9331,-20.9971,6.14,6.14,100,21.54,IS,1738794731


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display



# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    geo=True,
    size= 'Humidity',
    scale= 1,
    tiles= 'OSM')

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df['Max Temp'] = pd.to_numeric(city_data_df['Max Temp'], errors= 'coerce')
ideal_city_df = city_data_df[city_data_df['Max Temp'] < 80].copy()
ideal_city_df = city_data_df[city_data_df['Humidity'] < 15]

# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,benghazi,32.1167,20.0667,14.43,14.43,99,5.28,LY,1738794437
3,3,port-aux-francais,-49.3500,70.2167,7.39,7.39,99,8.84,TF,1738794719
4,4,grytviken,-54.2811,-36.5092,1.62,1.62,98,4.36,GS,1738794720
5,5,shenjiamen,29.9576,122.2980,3.65,3.65,6,3.40,CN,1738794721
8,8,hun,29.1268,15.9477,13.53,13.53,99,2.76,LY,1738794724


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,benghazi,32.1167,20.0667,14.43,14.43,99,5.28,LY,1738794437,
3,3,port-aux-francais,-49.3500,70.2167,7.39,7.39,99,8.84,TF,1738794719,
4,4,grytviken,-54.2811,-36.5092,1.62,1.62,98,4.36,GS,1738794720,
5,5,shenjiamen,29.9576,122.2980,3.65,3.65,6,3.40,CN,1738794721,
8,8,hun,29.1268,15.9477,13.53,13.53,99,2.76,LY,1738794724,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 5000
params = {
    'categories' : 'accommodation.hotel',
    'limit' : 10,
    'apiKey' : geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lati = row['Lat']
    long = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{long},{lati},{radius}'
    params["bias"] = f'proximity:{long},{lati}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    

    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError) as e:
        # If no hotel is found, set the hotel name as "No hotel found".
        print(e)
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    #Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
benghazi - nearest hotel: ترجاب
port-aux-francais - nearest hotel: Keravel
list index out of range
grytviken - nearest hotel: No hotel found
shenjiamen - nearest hotel: Zhonghan Hotel
hun - nearest hotel: فندق الهروج
list index out of range
ouled mimoun - nearest hotel: No hotel found
selfoss - nearest hotel: Miðgarður
hermanus - nearest hotel: Aloe guest house
list index out of range
ta'izz - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
la passe - nearest hotel: L'Estuaire
isafjordur - nearest hotel: Hótel Horn
wuhu - nearest hotel: Crowne Plaza Hotel Wuhu
corte - nearest hotel: Armonia
camarinas - nearest hotel: O Parranda
invercargill - nearest hotel: The Grand
bermeo - nearest hotel: Aldatzeta ostatua
harstad - nearest hotel: Scandic Harstad
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
list index out of range
papatowai - nearest hotel: No hotel found
midvagur - nearest hotel: GiljaHome
lucheng - nearest hotel:

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,benghazi,32.1167,20.0667,14.43,14.43,99,5.28,LY,1738794437,ترجاب
3,3,port-aux-francais,-49.3500,70.2167,7.39,7.39,99,8.84,TF,1738794719,Keravel
4,4,grytviken,-54.2811,-36.5092,1.62,1.62,98,4.36,GS,1738794720,No hotel found
5,5,shenjiamen,29.9576,122.2980,3.65,3.65,6,3.40,CN,1738794721,Zhonghan Hotel
8,8,hun,29.1268,15.9477,13.53,13.53,99,2.76,LY,1738794724,فندق الهروج
...,...,...,...,...,...,...,...,...,...,...,...
546,546,fauske,67.2588,15.3918,4.41,4.41,83,5.73,NO,1738795344,Scandic Fauske
553,553,saint-francois,46.4154,3.9054,3.69,3.15,100,0.00,FR,1738795351,No hotel found
566,566,ayr,55.4627,-4.6339,6.95,3.99,20,2.57,GB,1738795366,The Berkeley
571,571,laguna,38.4210,-121.4238,13.90,12.08,0,0.89,US,1738795372,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    x = 'Lng',
    y = 'Lat',
    geo=True,
    hover_cols= ['Hotel Name'],
    scale= 1,
    tiles= 'OSM')

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name)